# Trade Study Evaluations

NOTE: work in progress...

Now that we know how to make algorithms and how to make partial-stack AVs, we can leverage the simplicity of AVstack to initiate some trade studies. In these trade studies, we will be comparing and contrasting different designs against objective performance metrics on scenes.

In [1]:
import os
import avstack
import avapi
from tqdm import tqdm

%load_ext autoreload
%autoreload 2

data_base = '../../lib-avstack-api/data/'
obj_data_dir_k = os.path.join(data_base, 'KITTI/object')
raw_data_dir_k = os.path.join(data_base, 'KITTI/raw')
obj_data_dir_n = os.path.join(data_base, 'nuScenes')

KSM = avapi.kitti.KittiScenesManager(obj_data_dir_k, raw_data_dir_k, convert_raw=False)
KDM = KSM.get_scene_dataset_by_index(scene_idx=0)

NSM = avapi.nuscenes.nuScenesManager(obj_data_dir_n)
NDM = NSM.get_scene_dataset_by_index(0)

Cannot import rss library


In [2]:
def run_our_av(DM, av):
    """Run an AV through a sequence of data"""
    data_manager = avstack.datastructs.DataManager(max_size=1)  # we either pop or we lose it
    for frame in tqdm(DM.frames):
        # -- add data --> sensor data has ID attached, so it knows where to go
        data_manager.push(DM.get_lidar(frame, sensor="main_lidar"))
        data_manager.push(DM.get_image(frame, sensor="main_camera"))
        
        # -- run next frame
        t = DM.framerate * frame
        av.tick(frame, t, data_manager)

t_init = 0
ego_init = None  # we don't need this for an agent without localization
result_folder = 'results_AV'